In [84]:
import keras
from keras.datasets import cifar10
import numpy as np 
import itertools
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import accuracy_score
from sklearn.neighbors import BallTree
import pickle
import glob
!pip install opencv-contrib-python==3.4.2.17
import cv2

In [85]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [86]:
print(x_train[0].shape)
# print(x_test[0])

(32, 32, 3)


In [0]:
def desSIFT(image):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)
    #draw keypoints
    #import matplotlib.pyplot as plt		
    #img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
    #plt.imshow(img2),plt.show()
    return kp,des

def describeORB( image):
    #An efficient alternative to SIFT or SURF
    #doc http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_orb/py_orb.html
    #ORB is basically a fusion of FAST keypoint detector and BRIEF descriptor 
    #with many modifications to enhance the performance
    orb=cv2.ORB_create()
    kp, des=orb.detectAndCompute(image,None)
    return kp,des

def describeSURF( image):
    surf = cv2.xfeatures2d.SURF_create()
    # it is better to have this value between 300 and 500
    surf.setHessianThreshold(400)
    kp, des = surf.detectAndCompute(image,None)
    return kp,des


In [0]:
def getDescriptors(images) : 
    descriptors = []
    
    for image in images : 
        print (image.shape)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        kp, des = describeSURF(image)
        if des is not None : 
            descriptors.append(des)
            
    # descriptors = list(itertools.chain.from_iterable(descriptors))
    descriptors = np.concatenate(descriptors, axis=0)
    descriptors = np.asarray(descriptors)
        
    return descriptors

def getVLADDescriptors(images, images_lables, visualDic):
    descriptors = []
    labels = []
    
    count = 0
    for image in images : 
        kp, des = describeSURF(image)
        if des is not None : 
            v = VLAD(des, visualDic)
            descriptors.append(v)
            labels.append(images_lables[count])
        count += 1
            
            
    # descriptors = list(itertools.chain.from_iterable(descriptors))
    # descriptors = np.concatenate(descriptors, axis=0)
    descriptors = np.asarray(descriptors)
        
    return descriptors, labels
    
def kMeans(training, k) : 
    est = KMeans(n_clusters = k, init = 'k-means++').fit(training)
    return est

def VLAD(X, visualDictionary) : 
    
    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels = visualDictionary.labels_
    k = visualDictionary.n_clusters
    
    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    return V


In [89]:
sift_des = getDescriptors(np.concatenate((x_train, x_test), axis = 0))
print (sift_des.shape)




Streaming output truncated to the last 5000 lines.
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(

In [0]:
# print (sift_des[50000][0])
visDic = MiniBatchKMeans(init='k-means++', n_clusters=50,max_iter=1000, batch_size=1000, n_init=10, max_no_improvement=10, verbose=0).fit(sift_des)

In [91]:
vlad_des, labels = getVLADDescriptors(x_train, y_train, visDic)
vlad_des_test, labels_test = getVLADDescriptors(x_test, y_test, visDic)
print  (vlad_des_test)
print (labels_test)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.0459024  -0.01153333  0.00830984 ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
[array([3]), array([8]), array([8]), array([0]), array([6]), array([0]), array([9]), array([7]), array([8]), array([5]), array([7]), array([0]), array([9]), array([2]), array([6]), array([5]), array([4]), array([9]), array([4]), array([5]), array([9]), array([7]), array([9]), array([0]), array([3]), array([8]), array([8]), array([7]), array([4]), array([6]), array([7]), array([2]), array([8]), array([0]), array([2]), array([8]), array([1]), array([7]), array([2]), array([7]), array([9]), array([0]), array([3]), array([8]), array([6]), array([6]), arr

In [92]:
clf = cv2.ml.KNearest_create()
clf.train(np.float32(vlad_des), cv2.ml.ROW_SAMPLE, np.float32(labels))

True

In [0]:
ret, results, neighbours ,dist = clf.findNearest(np.float32(vlad_des_test), k=10)

In [94]:
print (results)
print (labels_test)

[[3.]
 [8.]
 [8.]
 ...
 [6.]
 [6.]
 [7.]]
[array([3]), array([8]), array([8]), array([0]), array([6]), array([0]), array([9]), array([7]), array([8]), array([5]), array([7]), array([0]), array([9]), array([2]), array([6]), array([5]), array([4]), array([9]), array([4]), array([5]), array([9]), array([7]), array([9]), array([0]), array([3]), array([8]), array([8]), array([7]), array([4]), array([6]), array([7]), array([2]), array([8]), array([0]), array([2]), array([8]), array([1]), array([7]), array([2]), array([7]), array([9]), array([0]), array([3]), array([8]), array([6]), array([6]), array([4]), array([5]), array([4]), array([6]), array([2]), array([1]), array([3]), array([4]), array([3]), array([2]), array([2]), array([4]), array([2]), array([9]), array([7]), array([2]), array([6]), array([2]), array([2]), array([0]), array([5]), array([4]), array([6]), array([8]), array([5]), array([0]), array([9]), array([9]), array([3]), array([7]), array([5]), array([0]), array([3]), array([3]

In [95]:
pred_label = []
correct_label = []

for label in results:
    pred_label.append(int(label))

for label in labels_test:
    correct_label.append(int(label))

print (y_test)
print (pred_label)

print (len(pred_label))
print (len(correct_label))
    
accuracy_score(labels_test, pred_label)

[[3]
 [8]
 [8]
 ...
 [5]
 [1]
 [7]]
[3, 8, 8, 8, 4, 6, 0, 2, 5, 3, 5, 2, 9, 2, 6, 0, 0, 4, 7, 2, 4, 7, 2, 1, 7, 0, 8, 6, 5, 0, 4, 0, 8, 9, 5, 8, 0, 5, 3, 1, 0, 1, 0, 8, 9, 5, 4, 5, 2, 1, 7, 0, 7, 6, 3, 7, 2, 2, 3, 5, 1, 3, 9, 5, 8, 9, 1, 8, 3, 0, 6, 0, 3, 0, 7, 4, 0, 0, 0, 7, 0, 1, 1, 1, 0, 2, 0, 3, 2, 8, 2, 5, 2, 1, 0, 9, 7, 6, 8, 1, 0, 5, 8, 8, 9, 8, 4, 6, 7, 0, 0, 4, 6, 0, 4, 0, 1, 7, 4, 7, 8, 7, 2, 0, 0, 0, 0, 1, 5, 5, 4, 1, 1, 9, 6, 2, 6, 5, 9, 7, 0, 9, 3, 7, 9, 2, 0, 3, 6, 3, 0, 0, 0, 3, 0, 6, 0, 2, 8, 1, 8, 8, 7, 8, 1, 7, 0, 2, 0, 1, 1, 4, 7, 7, 6, 2, 5, 2, 2, 1, 5, 2, 0, 0, 5, 4, 4, 6, 1, 5, 0, 9, 6, 0, 0, 9, 3, 6, 8, 1, 4, 0, 8, 8, 0, 0, 5, 4, 0, 1, 2, 7, 0, 2, 4, 6, 3, 6, 6, 5, 1, 0, 1, 0, 0, 4, 8, 7, 7, 3, 0, 4, 4, 0, 2, 6, 3, 2, 7, 4, 0, 7, 4, 1, 3, 0, 6, 8, 2, 9, 4, 8, 6, 4, 9, 6, 0, 4, 9, 6, 1, 0, 9, 3, 8, 7, 0, 8, 6, 0, 5, 5, 9, 1, 5, 6, 1, 8, 0, 0, 0, 6, 4, 2, 2, 7, 7, 3, 0, 2, 7, 7, 2, 9, 3, 0, 3, 0, 8, 6, 8, 8, 1, 8, 5, 1, 8, 0, 5, 5, 1, 2, 0, 1, 7, 0, 9, 5, 7, 4, 0, 

0.24713584288052373